**LINGUAGENS E PARADIGMAS - PROVA FINAL**

**Prof. Luciano Silva**

*   A prova final consiste de duas questões de implementação de linguagens imperativas
*  A primeira questão abordará declarações e uso de variáveis com tipos
*  A segunda questão abordará comandos imperativos
*  Em cada questão você deverá implementar analisador léxico, classes para representar a árvore sintática, analisador sintático, visitor para tabela de símbolos, visitor para decoração de tipos, visitor para verificação de tipos e visitor para geração de código intermediário em P-CODE
* Durante a prova, o aluno poderá consultar os notebooks desenvolvidos em aula (inclusive esta revisão), diretamente no github da disciplina, e usar a implementação de referência abaixo
* Finalizada a prova, o aluno deverá submeter o notebook inteiro no link disponibilizado no Blackboard
* A prova final será monitorada pelo software Proctório


**IMPLEMENTAÇÃO DE REFERÊNCIA**

In [1]:
!pip install rply


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


* Analisador léxico...

In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

* Classes dos nós da árvore sintática, já com o método accept para receber os visitors:

In [3]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


* Implementação do analisador sintático:

In [4]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS','COMP','IF','ELSE','WHILE'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string")

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int")

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [5]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp


In [6]:
arvore=parser.parse(lexer.lex('int x;int y; x=3; while(x<6) if (x<3) x=1; else x=2;'))
arvore.accept(SymbolTable())
print(ST)

{'x': 'int', 'y': 'int'}


In [7]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"



In [8]:
arvore=parser.parse(lexer.lex('int x;int y; x=3; while(x<6) if (x<3) x=1; else x=2;'))
arvore.accept(SymbolTable())
arvore.accept(Decorator())

In [9]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')


In [10]:
arvore=parser.parse(lexer.lex('int x;int y; x=3; while(x<6) if (x<3) x=1; else x=2;'))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())

In [11]:
class IntermediateCode(Visitor):

  def __init__(self):
        self.level=0  #nível do if-then-else
        self.loop=0  #número do loop
        self.comp={"==":"8","!=":"9","<":"10", "<=":"11", ">":"12", ">=":"13" } #códigos dos comparadores


  def visit_prog(self, i):
        i.stmts.accept(self)

  def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)


  def visit_statement(self, d):
        d.cmd.accept(self)

  def visit_atrib(self, i):
        i.expr.accept(self)
        print("sto 0",i.id)

  def visit_ifelse(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        if i.ie2!=None:  # se houver else, gera o código
          i.ie2.accept(self)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_while(self, i):
        self.loop=self.loop+1
        print("loop"+str(self.loop)+":") #gera o rótulo da seção true
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp loop"+str(self.loop)) # pula para o início do loop
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1


  def visit_number(self, i):
    print("lit 0 ",i.value)

  def visit_id(self, i):
    print("lod 0 ",i.value)

  def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 2")


  def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 3")


  def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 4")


  def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 5")


In [12]:
arvore=parser.parse(lexer.lex('int x;int y; x=3; while(x<6) if (x<3) x=1; else x=2;'))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())

lit 0  3
sto 0 x
loop1:
lit 0  6
lod 0  x
opr 0 10
jpc 0 true1
jmp 0 final1
true1:
lit 0  3
lod 0  x
opr 0 10
jpc 0 true2
jmp 0 false2
true2:
lit 0  1
sto 0 x
jmp final2
false2:
lit 0  2
sto 0 x
final2:
jmp loop1
final1:


**QUESTÃO 1 (5.0 pontos)**

Nossa gramática imperativa (acima) permite declarações com tipos simples (int e string). Para produzir programas mais úteis, podemos permitir que o programador **defina variáveis do tipo int e já inicialize seu conteúdo com expressões**. Por exemplo:

int x = 3-4*5;

int y = 2*x-3;

Modifique a implementação dó código de referência para permitir este tipo de **declaração com inicialização.**

In [13]:
#ANALISADOR LÉXICO (0.5 PONTO)
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

In [14]:
#CLASSES DA ÁRVORE SINTÁTICA (1.0 PONTO)

#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp, expr):
        self.id = id
        self.tp = tp
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_vardecl(self)


class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [15]:
#ANALISADOR SINTÁTICO (1.0 PONTO)
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS','COMP','IF','ELSE','WHILE'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string", None)

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)


@pg.production('vardecl : INT ID EQUALS expression SEMICOL')
def vardecl_int_expr(p):
    return VarDecl(p[1].getstr(), "int", p[3])

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [16]:
# VISITOR PARA TABELA DE SÍMBOLOS (0.5 PONTO)
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp


In [17]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"



In [18]:
#VISITOR PARA DECORAÇÃO E VERIFICAÇÃO DE TIPOS (1.0 PONTO)

# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_vardecl(self, d):
        if d.expr != None:
          if d.decor_type != d.expr.decor_type:
            raise AssertionError('type error')


    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')


In [19]:
#VISITOR PARA GERAR CÓDIGO INTERMEDIÁRIO (1.0 PONTO)

class IntermediateCode(Visitor):

  def __init__(self):
        self.level=0  #nível do if-then-else
        self.loop=0  #número do loop
        self.comp={"==":"8","!=":"9","<":"10", "<=":"11", ">":"12", ">=":"13" } #códigos dos comparadores


  def visit_prog(self, i):
        i.decls.accept(self)
        i.stmts.accept(self)

  def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)


  def visit_statement(self, d):
        d.cmd.accept(self)

  def visit_vardecls(self, i):
        i.decl.accept(self)
        if i.decls != None:
          i.decls.accept(self)

  def visit_vardecl(self, i):
        if i.expr != None:
          i.expr.accept(self)
          print("sto 0",i.id)

  def visit_atrib(self, i):
        i.expr.accept(self)
        print("sto 0",i.id)

  def visit_ifelse(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        if i.ie2!=None:  # se houver else, gera o código
          i.ie2.accept(self)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_while(self, i):
        self.loop=self.loop+1
        print("loop"+str(self.loop)+":") #gera o rótulo da seção true
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp loop"+str(self.loop)) # pula para o início do loop
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1


  def visit_number(self, i):
    print("lit 0 ",i.value)

  def visit_id(self, i):
    print("lod 0 ",i.value)

  def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 2")


  def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 3")


  def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 4")


  def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 5")


In [20]:
arvore=parser.parse(lexer.lex('int x; int y; int z= 2 + 3 + 5 - 7; x=4;'))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())

lit 0  2
lit 0  3
opr 0 2
lit 0  5
opr 0 2
lit 0  7
opr 0 3
sto 0 z
lit 0  4
sto 0 x


**QUESTÃO 2 (5.0 pontos)**

Uma maneira mais compacta de escrever comandos condicionais cujo resultado final é uma atribuição para uma variáveis é através da construção **if-ternário**. Por exemplo, considere o seguinte comando if:

**if** (x==3):
*   y=4;

**else**:
* y=5;

Usando o mecanismo de if-ternário, teríamos:

**y** = **x==3** ? **4** : **5** ;

Antes do sinal de ?, temos a condição. Se esta condição for verdadeira, atribuiremos a expressão que está entre ? e : à variável. Caso contrário, atribuíremos a expressão entre : e ; à varíavel.


Modifique a implementação dó código de referência para permitir ifs-ternários dentro do programa.


In [21]:
#ANALISADOR LÉXICO (0.5 PONTO)
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')
lg.add('QMARK', r'\?')
lg.add('COL', r':')

lg.ignore('\s+')

lexer = lg.build()

In [22]:
#CLASSES DA ÁRVORE SINTÁTICA (1.0 PONTO)

#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp, expr):
        self.id = id
        self.tp = tp
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_vardecl(self)


class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class AtribTern(BaseBox):
    def __init__(self, id,expr1, comp, expr2, r1, r2):
        self.id = id
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.r1=r1
        self.r2=r2

    def accept(self, visitor):
        visitor.visit_atribtern(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [23]:
#ANALISADOR SINTÁTICO (1.0 PONTO)
#ANALISADOR SINTÁTICO (1.0 PONTO)
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS','COMP','IF','ELSE','WHILE', 'QMARK', 'COL'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string", None)

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)


@pg.production('vardecl : INT ID EQUALS expression SEMICOL')
def vardecl_int_expr(p):
    return VarDecl(p[1].getstr(), "int", p[3])

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : ID EQUALS expression COMP expression QMARK expression COL expression SEMICOL')
def statement_atrib_tern(p):
    return AtribTern(p[0].getstr(),p[2], p[3], p[4], p[6], p[8])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : ID EQUALS expression COMP expression QMARK expression COL expression SEMICOL')
def statement_atrib_tern(p):
    return AtribTern(p[0].getstr(),p[2], p[3], p[4], p[6], p[8])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [24]:
#VISITOR PARA TABELA DE SÍMBOLOS (0.5 PONTO)
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp


In [25]:
#VISITOR PARA DECORAÇÃO E VERIFICAÇÃO DE TIPOS (1.0 PONTO)
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_atribtern(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.r1.accept(self)
        i.r2.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"



In [26]:
#VISITOR PARA DECORAÇÃO E VERIFICAÇÃO DE TIPOS (1.0 PONTO)

# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_vardecl(self, d):
        if d.expr != None:
          if d.decor_type != d.expr.decor_type:
            raise AssertionError('type error')


    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_atribtern(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')


In [27]:
#VISITOR PARA GERAR CÓDIGO INTERMEDIÁRIO (1.0 PONTO)

#VISITOR PARA GERAR CÓDIGO INTERMEDIÁRIO (1.0 PONTO)

class IntermediateCode(Visitor):

  def __init__(self):
        self.level=0  #nível do if-then-else
        self.loop=0  #número do loop
        self.comp={"==":"8","!=":"9","<":"10", "<=":"11", ">":"12", ">=":"13" } #códigos dos comparadores


  def visit_prog(self, i):
        i.decls.accept(self)
        i.stmts.accept(self)

  def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)


  def visit_statement(self, d):
        d.cmd.accept(self)

  def visit_vardecls(self, i):
        i.decl.accept(self)
        if i.decls != None:
          i.decls.accept(self)

  def visit_vardecl(self, i):
        if i.expr != None:
          i.expr.accept(self)
          print("sto 0",i.id)

  def visit_atrib(self, i):
        i.expr.accept(self)
        print("sto 0",i.id)

  def visit_atribtern(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.r1.accept(self) # gera o código para true
        print("sto 0",i.id)
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        i.r2.accept(self)
        print("sto 0",i.id)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_ifelse(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        if i.ie2!=None:  # se houver else, gera o código
          i.ie2.accept(self)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_while(self, i):
        self.loop=self.loop+1
        print("loop"+str(self.loop)+":") #gera o rótulo da seção true
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp loop"+str(self.loop)) # pula para o início do loop
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1


  def visit_number(self, i):
    print("lit 0 ",i.value)

  def visit_id(self, i):
    print("lod 0 ",i.value)

  def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 2")


  def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 3")


  def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 4")


  def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 5")


In [28]:
arvore=parser.parse(lexer.lex('int x; int y; x=4; y = x==3 ? 4 : 5 ;'))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())

lit 0  4
sto 0 x
lit 0  3
lod 0  x
opr 0 8
jpc 0 true1
jmp 0 false1
true1:
lit 0  4
sto 0 y
jmp final1
false1:
lit 0  5
sto 0 y
final1:
